In [ ]:
import os, shutil
from matplotlib.image import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import Model, Input
from keras import optimizers
from keras.layers import Conv2D, Activation, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras import callbacks
from tensorflow import keras
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from tensorflow.keras.utils import plot_model

In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'C:/0_thesis/0_dataset-analysis')

from age_groups import *

In [ ]:
ds_path = 'C:/0_thesis/dataset/utkface-wild-pepper/'
csv_path = 'C:/0_thesis/dataset/utkface-wild-pepper.csv'
id_process = "19biaswild"
results_folder = "C:/0_thesis/2_model/TESTING/both/"+str(id_process)

pepper_val = "C:/0_thesis/dataset/pepper-validation-data"
pepper_val_csv = "C:/0_thesis/dataset/pepper-validation-data.csv"

In [ ]:
batch_size = 64 # !!

img_size = 256
x_col = 'filename'
y_col = ['age', 'gender']

## EVALUATE ON same validation set used in training

In [ ]:
validation_data = pd.read_csv(results_folder+"/validation_data.csv")
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(validation_data, 
                                                directory = ds_path, 
                                                x_col = x_col, 
                                                y_col = y_col, 
                                                target_size = (img_size, img_size),
                                                class_mode="multi_output",
                                                shuffle = False,
                                                batch_size = batch_size)

## ON PEPPER PHOTOS

In [ ]:
# VAL ON PEPPER PHOTOS
validation_data = pd.read_csv(pepper_val_csv)

gender_mapper = {'male': 0, 'female': 1}
validation_data = validation_data.replace({"gender": gender_mapper})

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(validation_data, 
                                                directory = pepper_val, 
                                                x_col = x_col, 
                                                y_col = y_col, 
                                                target_size = (img_size, img_size),
                                                class_mode="multi_output",
                                                shuffle=False,
                                                batch_size = batch_size)

In [ ]:
np.bincount(validation_data["gender"])

## Load and evaluate the model

In [ ]:
model = keras.models.load_model(results_folder+"/model")

In [ ]:
# Evaluate
model.evaluate(val_generator)

In [ ]:
# Predict
prediction = model.predict(val_generator)
y_pred_age = np.round(prediction[0])
#y_pred_age = prediction[0].argmax(axis=-1)
y_pred_gender = np.round(prediction[1])

y_pred_gender = y_pred_gender.astype('int')
y_pred_age = y_pred_age.astype('int')
validation_data["gender"]=validation_data["gender"].astype(int)
validation_data["age"]=validation_data["age"].astype(int)

In [ ]:
precision = tf.keras.metrics.Precision()
precision.update_state(validation_data["gender"], y_pred_gender)
print("Precision on gender: ", precision.result().numpy())
precision.update_state(validation_data["age"], y_pred_age)
print("Precision on age: ", precision.result().numpy())

In [ ]:
recall = tf.keras.metrics.Recall()
recall.update_state(validation_data["gender"], y_pred_gender)
print("Recall on gender: ", recall.result().numpy())
recall.update_state(validation_data["age"], y_pred_age)
print("Recall on age: ", recall.result().numpy())

In [ ]:
# Confusion matrix GENDER
cm = confusion_matrix(validation_data['gender'], y_pred_gender, labels=np.unique(y_pred_gender))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_pred_gender))
disp.plot()
#plt.savefig(results_folder+"/cm_gender.jpg")

In [ ]:
y_pred_groups = []

for pred in y_pred_age:
    y_pred_groups.append(AgeGroups().getGroupFromAge(pred))
    
cm = confusion_matrix(validation_data["age-group"], y_pred_groups)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_pred_groups))
disp.plot()
#plt.savefig(results_folder+"/cm_age-groups.jpg")

In [ ]:
# Print 9 random images with PREDICTION
import random

plt.figure(figsize=(10,10))

indices = random.sample(np.arange(0,len(validation_data.index)).tolist(),9)

for j, i in enumerate(indices):
    sample = validation_data.iloc[i]
    
    actual_gender = "Female" if sample.gender==1 else "Male"
    pred_gender = "Female" if y_pred_gender[i]==1 else "Male"
    actual_age = sample['age']
    pred_age = y_pred_age[i]
    
    plt.subplot(3,3,j+1)
    plt.axis('off')
    plt.title('Actual: %s, %s\nPred: %s, %s' % (actual_gender, actual_age, pred_gender, pred_age))
    #plt.imshow(Image.open(ds_path+"/"+sample.filename))
    plt.imshow(Image.open(pepper_val+"/"+sample.filename))

plt.savefig(results_folder+"/example_pepper.jpg")
plt.show()

## ON EXTERNAL IMAGES

In [ ]:
id_process = "19wildker"
results_folder = "C:/0_thesis/2_model/TESTING/both/"+str(id_process)

!python 4_predict.py --modelpath=$results_folder --inputdir="C:/0_thesis/2_model/TESTING/ds-test"